# Installation & Import

## Install

In [ ]:
!pip install langchain==0.3.24 langchain-core==0.3.56 langchain-community==0.3.23 langchain-groq==0.3.2 langchain-qdrant==0.2.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 26.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.5/127.5 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.7/327.7 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.5 MB/s eta 0:00:00


In [ ]:
!pip install sentence_transformers==3.4.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 46.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 29.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 77.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [ ]:
!pip install gdown==5.2.0

In [ ]:
!pip install googletrans==4.0.2

## Import

In [ ]:
import ipywidgets as widgets
from IPython.display import display, Markdown, clear_output, HTML, Javascript
from google.colab import output

In [ ]:
import os
import pickle
import sys
import json
import re
import random
import asyncio
import time
import shutil
import uuid
import psutil
import requests
from tqdm import tqdm
import numpy as np
import pandas as pd

In [ ]:
from langchain_community.embeddings import HuggingFaceEmbeddings

from langchain.llms import HuggingFacePipeline
#from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA

from langchain_core.prompts import ChatPromptTemplate
from langchain.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.output_parsers import JsonOutputParser

from langchain_groq import ChatGroq

#from langchain.vectorstores import Chroma
from langchain_qdrant import QdrantVectorStore
from langchain_qdrant import Qdrant  # Use the new Qdrant class

from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain

from qdrant_client import QdrantClient
from qdrant_client.models import PointStruct, VectorParams, Distance

In [ ]:
import torch
from transformers import pipeline, AutoModelForSequenceClassification, AutoTokenizer
from googletrans import Translator
import google.generativeai as genai

# Functions

In [ ]:
# Define allowed characters: Persian, English, digits, spaces, punctuation, and ZWNJ
RE_PATTERN = re.compile(r'^[\u0600-\u06FF\uFB50-\uFDFF\u200Ca-zA-Z0-9\s?!.,…]+$')

def is_persian_or_english(text):
    """
    Checks if the given text contains only Persian or English characters.
    Allowed characters:
      - Persian (U+0600–U+06FF, U+FB50–U+FDFF)
      - English (A-Z, a-z)
      - Numbers (0-9)
      - Spaces, punctuation (?, !, ., …)
      - Persian zero-width non-joiner (ZWNJ, U+200C)

    Returns True if valid, otherwise False.
    """

    return bool(RE_PATTERN.match(text))

In [ ]:
# Example usage:
sample_text = "سلام سینا، خوشحالم که باهات ملاقات می‌کنم. من خوبم! Thank you?"
print(is_persian_or_english(sample_text))  # True

sample_text2 = "ی‌ک کتاب"  # Contains ZWNJ
print(is_persian_or_english(sample_text2))  # True (Correctly detects Persian)

sample_text3 = "你好，这是中文。"  # Chinese text
print(is_persian_or_english(sample_text3))  # False

True
True
False


# LLM API | Token

In [ ]:
# for getting information
GROQ_API_KEY = "your-groq-api-key"

# for translation
GEMINI_API_KEY = 'your-gemini-api-key'

# Sentiment Analysis, Roberta, BERT

In [ ]:
# 🔹 Load RoBERTa Model & Tokenizer
roberta_model_name = "cardiffnlp/twitter-roberta-base-sentiment"
roberta_tokenizer = AutoTokenizer.from_pretrained(roberta_model_name)
roberta_model = AutoModelForSequenceClassification.from_pretrained(roberta_model_name)

def get_sentiment_score_roberta(text):
    """Returns sentiment category and probabilities using RoBERTa model."""
    tokens = roberta_tokenizer(text, return_tensors="pt")
    output = roberta_model(**tokens)

    # Apply softmax to get probability distribution
    scores = torch.nn.functional.softmax(output.logits, dim=-1)[0].tolist()

    # Define sentiment categories
    categories = ["negative", "neutral", "positive"]

    # Assign category based on highest probability
    label = categories[scores.index(max(scores))]

    return {
        "label": label,
        "score": {
            "neg": scores[0],
            "neu": scores[1],
            "pos": scores[2]
        }
    }


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/747 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

In [ ]:
# Load BERT Model & Tokenizer (Multilingual Sentiment Model)
bert_model_name = "nlptown/bert-base-multilingual-uncased-sentiment"
bert_tokenizer = AutoTokenizer.from_pretrained(bert_model_name)
bert_model = AutoModelForSequenceClassification.from_pretrained(bert_model_name)

def get_sentiment_score_bert(text):
    """Returns sentiment category and probabilities using BERT model."""
    tokens = bert_tokenizer(text, return_tensors="pt")
    output = bert_model(**tokens)

    # Apply softmax to get probability distribution
    scores = torch.nn.functional.softmax(output.logits, dim=-1)[0].tolist()

    # BERT Sentiment model has 5 labels (1 to 5 stars)
    categories = ["negative", "negative", "neutral", "positive", "positive"]

    # Assign category based on the highest probability
    highest_index = scores.index(max(scores))  # Get index of highest probability (0 to 4)
    label = categories[highest_index]  # Map index to sentiment

    return {
        "label": label,
        "score": {
            "neg": scores[0] + scores[1],  # Sum probabilities for negative sentiment (1 & 2 stars)
            "neu": scores[2],  # Neutral (3 stars)
            "pos": scores[3] + scores[4]  # Sum probabilities for positive sentiment (4 & 5 stars)
        }
    }

In [ ]:
# Load the sentiment analysis pipeline with the multilingual BERT model
sentiment_pipeline_bert = pipeline("sentiment-analysis", model="nlptown/bert-base-multilingual-uncased-sentiment")

def sentiment_process(texts):
    """Process a list of texts using sentiment analysis and return a list of results."""
    if isinstance(texts, str):  # If a single string is passed, convert it to a list
        texts = [texts]

    results = sentiment_pipeline_bert(texts)  # Batch processing

    # Convert results into a structured list of dictionaries
    sentiment_results = [
        {"label": int(res["label"][0]), "score": res["score"]}
        for res in results
    ]

    return sentiment_results

def sentiment_process_batch(texts, batch_size=10):
    """Process a list of texts using sentiment analysis with progress tracking."""
    if isinstance(texts, str):  # If a single string is passed, convert it to a list
        texts = [texts]

    sentiment_results = []

    # Process texts in batches to track progress
    for i in tqdm(range(0, len(texts), batch_size), desc="Processing", unit="batch"):
        batch = texts[i:i + batch_size]  # Get the next batch
        results = sentiment_pipeline_bert(batch)  # Process batch

        # Convert results into a structured list of dictionaries
        sentiment_results.extend(
            [{"label": int(res["label"][0]), "score": res["score"]} for res in results]
        )

    return sentiment_results


# Test - GROQ HTTP

In [ ]:
def get_chat_completion(api_key, model, prompt):

    url = "https://api.groq.com/openai/v1/chat/completions"

    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }

    data = {
        "model": model,
        "messages": [{"role": "user", "content": prompt}]
    }

    response = requests.post(url, headers=headers, json=data)

    if response.status_code == 200:
      result = response.json()
      output = result["choices"][0]["message"]["content"]
      return response.headers, output
    else:
      response.raise_for_status()


def prompt(text, model='llama-3.1-8b-instant'):

  headers, response = get_chat_completion(GROQ_API_KEY, model, text)
  """
  print("limit-requests:", headers["x-ratelimit-limit-requests"])
  print("limit-tokens:", headers["x-ratelimit-limit-tokens"])
  print("remaining-requests:", headers["x-ratelimit-remaining-requests"])
  print("remaining-tokens:", headers["x-ratelimit-remaining-tokens"])
  print("reset-requests:", headers["x-ratelimit-reset-requests"])
  print("reset-tokens:", headers["x-ratelimit-reset-tokens"])

  print("-------------------------------------------------------\n")
  """
  # print(headers)
  return response

In [ ]:
def get_sentiment_prompt(text):
    txt_prompt = f"""Analyze the sentiment of the following text and return the result strictly in this format:

[{{'label': '<star rating>', 'score': <confidence_score>}}]

Guidelines:
- The score is a floating number between 0 and 1.
- 0 means TOTAL negative, and 1 means TOTAL positive.
- Categorize the text based on the score as follows:
  - 0.00 - 0.20 → 1 star: Very Negative
  - 0.21 - 0.40 → 2 stars: Negative
  - 0.41 - 0.60 → 3 stars: Neutral
  - 0.61 - 0.80 → 4 stars: Positive
  - 0.81 - 1.00 → 5 stars: Very Positive

The confidence score must be between 0 and 1. No extra text, explanation, or formatting should be included—only the structured JSON response.

Text:
"{text}"
"""
    return prompt(txt_prompt, 'llama-3.3-70b-versatile')

def get_sentiment_prompt2(text):
  txt_prompt = f"""Analyze the sentiment of the following text and return the result strictly in this format:

  [{{'label': '<sentiment_category>', 'score': {{'neg': <negative_score>, 'neu': <neutral_score>, 'pos': <positive_score>}}}}]

  Guidelines:
  - The sentiment category must be one of: **"negative", "neutral", "positive"**.
  - The score must be a dictionary with three keys: **'neg' (negative), 'neu' (neutral), 'pos' (positive)**.
  - Each score is a floating number between **0 and 1**.
  - The three scores **must sum to 1**.
  - Categorize the text sentiment based on these rules:
    - If **negative > 0.50** → label as `"negative"`
    - If **positive > 0.50** → label as `"positive"`
    - Otherwise → label as `"neutral"`

  Scoring Rule:
  - **If highly negative (0.00 - 0.20 neg)** → `neg` takes most of the weight.
  - **If highly positive (0.80 - 1.00 pos)** → `pos` takes most of the weight.
  - **If neutral (0.40 - 0.60 neu)** → Distribute weights accordingly.

  The response must be a **strict JSON format**, without extra text, explanation, or formatting.

  Text:
  "{text}"
  """

  return prompt(txt_prompt, 'llama-3.3-70b-versatile')

In [ ]:
def translate_using_groq(text, lang_src, lang_target):
    """
    Generates an English prompt for translating a given text from the source language to the target language.
    The translation must be clear, simple, and conversational.
    All words, including names, must be translated.
    Ensure the translated content has correct meaning and does not contain errors.
    Verify the grammar of both the source and target languages to ensure no structural or semantic issues.
    Do not add any extra symbols, characters, or explanations—return only the translated text.
    """
    pr = (
        f"Translate the following text from {lang_src} to {lang_target}. "
        "Translate all words, including names. "
        "Ensure the translation is clear, simple, and conversational. "
        "Verify that the translated content is meaningful and free of errors. "
        "Check the grammar of both the source and target languages to ensure no structural or semantic mistakes. "
        "Return only the translated text—do not add anything else.\n"
        f"Text: {text}"
    )

    return prompt(pr, 'gemma2-9b-it')


# Translate

## Google Translate

In [ ]:
# Initialize the translator variable to be used later
translator = Translator()
# Send a welcome message

LANGUAGES = {
    'af': 'afrikaans',
    'sq': 'albanian',
    'am': 'amharic',
    'ar': 'arabic',
    'hy': 'armenian',
    'az': 'azerbaijani',
    'eu': 'basque',
    'be': 'belarusian',
    'bn': 'bengali',
    'bs': 'bosnian',
    'bg': 'bulgarian',
    'ca': 'catalan',
    'ceb': 'cebuano',
    'ny': 'chichewa',
    'zh-cn': 'chinese',
    'co': 'corsican',
    'hr': 'croatian',
    'cs': 'czech',
    'da': 'danish',
    'nl': 'dutch',
    'en': 'english',
    'eo': 'esperanto',
    'et': 'estonian',
    'tl': 'filipino',
    'fi': 'finnish',
    'fr': 'french',
    'fy': 'frisian',
    'gl': 'galician',
    'ka': 'georgian',
    'de': 'german',
    'el': 'greek',
    'gu': 'gujarati',
    'ht': 'haitian creole',
    'ha': 'hausa',
    'haw': 'hawaiian',
    'iw': 'hebrew',
    'he': 'hebrew',
    'hi': 'hindi',
    'hmn': 'hmong',
    'hu': 'hungarian',
    'is': 'icelandic',
    'ig': 'igbo',
    'id': 'indonesian',
    'ga': 'irish',
    'it': 'italian',
    'ja': 'japanese',
    'jw': 'javanese',
    'kn': 'kannada',
    'kk': 'kazakh',
    'km': 'khmer',
    'ko': 'korean',
    'ku': 'kurdish',
    'ky': 'kyrgyz',
    'lo': 'lao',
    'la': 'latin',
    'lv': 'latvian',
    'lt': 'lithuanian',
    'lb': 'luxembourgish',
    'mk': 'macedonian',
    'mg': 'malagasy',
    'ms': 'malay',
    'ml': 'malayalam',
    'mt': 'maltese',
    'mi': 'maori',
    'mr': 'marathi',
    'mn': 'mongolian',
    'my': 'myanmar',
    'ne': 'nepali',
    'no': 'norwegian',
    'or': 'odia',
    'ps': 'pashto',
    'fa': 'persian',
    'pl': 'polish',
    'pt': 'portuguese',
    'pa': 'punjabi',
    'ro': 'romanian',
    'ru': 'russian',
    'sm': 'samoan',
    'gd': 'scots gaelic',
    'sr': 'serbian',
    'st': 'sesotho',
    'sn': 'shona',
    'sd': 'sindhi',
    'si': 'sinhala',
    'sk': 'slovak',
    'sl': 'slovenian',
    'so': 'somali',
    'es': 'spanish',
    'su': 'sundanese',
    'sw': 'swahili',
    'sv': 'swedish',
    'tg': 'tajik',
    'ta': 'tamil',
    'te': 'telugu',
    'th': 'thai',
    'tr': 'turkish',
    'uk': 'ukrainian',
    'ur': 'urdu',
    'ug': 'uyghur',
    'uz': 'uzbek',
    'vi': 'vietnamese',
    'cy': 'welsh',
    'xh': 'xhosa',
    'yi': 'yiddish',
    'yo': 'yoruba',
    'zu': 'zulu'
}

def translate_text(text, lang="fa"):
  # Detect which language the user's input is written in and store it
  detectedLang = translator.detect(text)
  """"
  Since the detect function stores the language and confidence level,
  we want to only store the language that was detected
  """
  originalLang = (detectedLang.lang)
  translatedStatement = translator.translate(text, dest=lang, src=originalLang)
  return translatedStatement.text

def translate_text_to(text, src="fa", dest="en"):

  translatedStatement = translator.translate(text, dest=dest, src=src)
  return translatedStatement.text

In [ ]:
async def translate_text_async(text, lang="fa"):
  # Detect the language asynchronously
  detectedLang = await translator.detect(text)

  # Store only the detected language
  originalLang = detectedLang.lang

  # Translate asynchronously
  translatedStatement = await translator.translate(text, dest=lang, src=originalLang)

  return translatedStatement.text

async def translate_text_to_async(text, src="fa", dest="en"):
  translatedStatement = await translator.translate(text, dest=dest, src=src)
  return translatedStatement.text

In [ ]:
await translator.translate("سلام من سینا هستم", dest="en", src="fa")

## Gemini

In [ ]:
genai.configure(api_key=GEMINI_API_KEY)

# Create the model
generation_config = {
  "temperature": 1,
  "top_p": 0.95,
  "top_k": 40,
  "max_output_tokens": 8192,
  "response_mime_type": "text/plain",
}

model = genai.GenerativeModel(
  model_name="gemini-1.5-flash-8b", # gemini-2.0-flash
  generation_config=generation_config,
)

chat_session = model.start_chat(
  history=[
  ]
)

def generate_translation_prompt(text: str, source_lang: str, target_lang: str) -> str:
    """
    Generates a concise prompt for translating text from source_lang to target_lang,
    ensuring the response contains only the translation.

    Args:
        text (str): The text to be translated.
        source_lang (str): The source language.
        target_lang (str): The target language.

    Returns:
        str: The formatted prompt for translation.
    """
    return (
        f"Translate from {source_lang} to {target_lang}:\n\n"
        f"{text}\n\n"
        "Only return the translation without explanations or additional text."
    )

def translate_using_gemini(text: str, source_lang: str, target_lang: str):

    text = generate_translation_prompt(text, source_lang, target_lang)
    response = chat_session.send_message(text)
    return response.text

def call_gemini(text: str):
    response = chat_session.send_message(text)
    return response.text

In [ ]:
translate_using_gemini("سلام، من سینا هستم", "Farsi", "English")

'Hello, I am Sina.\n'

# Read and process Dataset frm CSV file

In [ ]:
load_vector_database = False

## Read Dataset

In [ ]:
temp_df = pd.read_csv('/content/300_dialog.csv', encoding='ISO-8859-1')
print(len(temp_df))

10864


In [ ]:
category_names = ['logical-appeal', 'emotion-appeal', 'credibility-appeal', 'foot-in-the-door',
           'self-modeling', 'personal-story', 'donation-information',
           'source-related-inquiry', 'task-related-inquiry', 'personal-related-inquiry']

df = temp_df[ temp_df['B4'] == 0 ]
df = df[['Unit', 'er_label_1']]
df = df[df['er_label_1'].isin(category_names)]
print(len(df))

3437


In [ ]:
df

,Unit,er_label_1
5,Are you involved with charities?,task-related-inquiry
18,Do you currently donate to your charity?,task-related-inquiry
22,Save The Children is an international non-gove...,credibility-appeal
28,These children really need the assistance.,logical-appeal
34,Do you have any more questions for me?,task-related-inquiry
...,...,...
10818,Maybe $0.50.,self-modeling
10835,Save the children.,logical-appeal
10836,dont you like children?,personal-related-inquiry
10837,they need your help.,logical-appeal


In [ ]:
df_grouped = {label: df1_group for label, df1_group in df.groupby('er_label_1')}

print("Total Categories:", len(df_grouped))

categories = list(df_grouped.keys())
print("Categories:", categories)

for key, value in df_grouped.items():
    print(f"Category: {key}, Count: {len(value)}")

Total Categories: 10
Categories: ['credibility-appeal', 'donation-information', 'emotion-appeal', 'foot-in-the-door', 'logical-appeal', 'personal-related-inquiry', 'personal-story', 'self-modeling', 'source-related-inquiry', 'task-related-inquiry']
Category: credibility-appeal, Count: 1083
Category: donation-information, Count: 491
Category: emotion-appeal, Count: 377
Category: foot-in-the-door, Count: 162
Category: logical-appeal, Count: 469
Category: personal-related-inquiry, Count: 168
Category: personal-story, Count: 153
Category: self-modeling, Count: 163
Category: source-related-inquiry, Count: 180
Category: task-related-inquiry, Count: 191


## Convert Dataset to Documents

In [ ]:
# Convert DataFrame to text & category lists
texts = df["Unit"].tolist()  # The actual text content
categories = df["er_label_1"].tolist()  # Categories as metadata

# Text Splitting (if necessary for large texts)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
docs = text_splitter.create_documents(texts)

# Checking the number of documents
print(f"Total number of documents: {len(docs)}")

# Assign metadata (category)
for i, doc in enumerate(docs):
    doc.metadata = {"category": categories[i]}

Total number of documents: 3437


In [ ]:
# Displaying document content and metadata
for i, doc in enumerate(docs):
    print(f"Document {i+1}: {doc.page_content} | Category: {doc.metadata['category']}")

Document 1: Are you involved with charities? | Category: task-related-inquiry
Document 2: Do you currently donate to your charity? | Category: task-related-inquiry
Document 3: Save The Children is an international non-governmental organization that promotes children's rights, provides relief and helps support children in developing countries. | Category: credibility-appeal
Document 4: These children really need the assistance. | Category: logical-appeal
Document 5: Do you have any more questions for me? | Category: task-related-inquiry
Document 6: How much money do you spend daily on extras like a coffee or treat? | Category: personal-related-inquiry
Document 7: Do you think that amount of money would make a difference in a needy child's life? | Category: task-related-inquiry
Document 8: Are you familiar with a charity called Save the Children? | Category: source-related-inquiry
Document 9: Save the Children is an international non-governmental organization that promotes children's rig

In [ ]:
!zip -r db_texts.zip /content/db_texts
!du -BM db_texts.zip | awk '{print $1+0}'

	zip warning: name not matched: /content/db_texts

zip error: Nothing to do! (try: zip -r db_texts.zip . -i /content/db_texts)
du: cannot access 'db_texts.zip': No such file or directory


# Download Dataset from Google Cloud

In [ ]:
load_vector_database = True

Download from gCloud

In [ ]:
!gdown --id "your-id"

/usr/local/lib/python3.11/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1kzySsjJbtzJ6IqYJL5_OgVVi4f8vL437
To: /content/db_texts.zip
100% 13.0M/13.0M [00:00<00:00, 47.6MB/s]


In [ ]:
!rm -rf /content/db_texts

!unzip /content/db_texts.zip -d /content

Archive:  /content/db_texts.zip
   creating: /content/db_texts/
 extracting: /content/db_texts/.lock  
   creating: /content/db_texts/collection/
   creating: /content/db_texts/collection/text_data_collection/
  inflating: /content/db_texts/collection/text_data_collection/storage.sqlite  
  inflating: /content/db_texts/meta.json  


# HuggingFace Embedding Model

In [ ]:
# Reinitialize embeddings
#model_name = "sentence-transformers/all-MiniLM-L6-v2"
model_name = "sentence-transformers/all-mpnet-base-v2"

try:
    torch.cuda.empty_cache()  # Clear GPU cache if any
    embeddings = HuggingFaceEmbeddings(
        model_name=model_name,
        model_kwargs={"device": "cpu"},
        encode_kwargs={"normalize_embeddings": True}
    )
    print("Embeddings reinitialized successfully!")
except Exception as e:
    print(f"Failed to reinitialize embeddings: {e}")
    raise

Embeddings reinitialized successfully!


# Qdrant Vector Database

## Functions

In [ ]:
def kill_qdrant_process():
    """Kill all Qdrant-related processes."""
    for proc in psutil.process_iter(['pid', 'name', 'cmdline']):
        try:
            cmdline = proc.info['cmdline'] or []
            if any('qdrant' in str(arg).lower() for arg in cmdline):
                print(f"Killing Qdrant process with PID: {proc.info['pid']}")
                proc.kill()
                proc.wait(timeout=5)
        except (psutil.NoSuchProcess, psutil.AccessDenied, psutil.ZombieProcess, psutil.TimeoutExpired):
            pass

def clear_qdrant_folder():
    """Clear the Qdrant storage folder entirely."""
    if os.path.exists(qdrant_path):
        print(f"Clearing Qdrant storage folder: {qdrant_path}")
        try:
            shutil.rmtree(qdrant_path)
            time.sleep(2)
        except OSError as e:
            print(f"Failed to clear folder: {e}")
    os.makedirs(qdrant_path, exist_ok=True)

def is_folder_accessible(path):
    """Check if the folder is accessible."""
    test_file = os.path.join(path, "test_access.txt")
    try:
        with open(test_file, 'w') as f:
            f.write("test")
        os.remove(test_file)
        return True
    except (OSError, IOError):
        return False

def clean_documents(docs):
    """Ensure documents are serializable by removing non-serializable attributes."""
    cleaned_docs = []
    for doc in docs:
        cleaned_doc = Document(
            page_content=str(doc.page_content),
            metadata={k: v for k, v in doc.metadata.items() if isinstance(v, (str, int, float, bool, list, dict))}
        )
        cleaned_docs.append(cleaned_doc)
    return cleaned_docs


## Create

In [ ]:
!rm -f /content/db_texts/lock

In [ ]:
# Qdrant storage path
qdrant_path = "/content/db_texts"

# Clean up existing client
if 'client' in globals() and client is not None:
    print("Deleting existing Qdrant client instance.")
    del client

# Aggressive cleanup
kill_qdrant_process()
clear_qdrant_folder()

# Verify folder accessibility
if not is_folder_accessible(qdrant_path):
    print("Storage folder is still inaccessible. Trying a new folder...")
    qdrant_path = "/content/db_texts_new"
    os.makedirs(qdrant_path, exist_ok=True)
    if not is_folder_accessible(qdrant_path):
        print("New folder is also inaccessible. Restarting the runtime is recommended.")
        raise RuntimeError("Cannot access Qdrant storage folder.")

# Create Qdrant client
try:
    client = QdrantClient(path=qdrant_path)
    print("New Qdrant client created successfully!")
except Exception as e:
    print(f"Failed to create Qdrant client: {e}")
    raise

Clearing Qdrant storage folder: /content/db_texts
New Qdrant client created successfully!


In [ ]:
# Clean documents
try:
    docs = clean_documents(docs)
    print("Documents cleaned successfully!")
except Exception as e:
    print(f"Failed to clean documents: {e}")
    raise

# Manually embed and insert documents
collection_name = "text_data_collection"
try:
    # Embed documents
    embedded_docs = embeddings.embed_documents([doc.page_content for doc in docs])
    print("Documents embedded successfully!")

    # Check if collection exists, create if it doesn't
    if not client.collection_exists(collection_name):
        client.create_collection(
            collection_name=collection_name,
            vectors_config=VectorParams(size=len(embedded_docs[0]), distance=Distance.COSINE)
        )
        print(f"Collection {collection_name} created successfully!")
    else:
        print(f"Collection {collection_name} already exists!")

    # Insert documents
    points = [
        PointStruct(
            id=str(uuid.uuid4()),
            vector=embedding,
            payload={"text": doc.page_content, "metadata": doc.metadata}
        )
        for doc, embedding in zip(docs, embedded_docs)
    ]
    client.upsert(collection_name=collection_name, points=points)
    print("Documents inserted into Qdrant successfully!")
except Exception as e:
    print(f"Failed to process Qdrant vector store manually: {e}")
    raise

# Initialize QdrantVectorStore (using langchain_qdrant)
try:
    qdrant = Qdrant(
        client=client,
        collection_name=collection_name,
        embeddings=embeddings  # Correct parameter name
    )
    print("Qdrant Vector Store initialized successfully!")
except Exception as e:
    print(f"Failed to initialize Qdrant Vector Store: {e}")
    raise

Embeddings reinitialized successfully!
Documents cleaned successfully!
Documents embedded successfully!
Collection text_data_collection created successfully!
Documents inserted into Qdrant successfully!
Qdrant Vector Store initialized successfully!


## Load

In [ ]:
# Aggressive cleanup
kill_qdrant_process()
clear_qdrant_folder()

Clearing Qdrant storage folder: /content/db_texts


In [ ]:
!rm -rf /content/db_texts
!unzip /content/db_texts.zip -d /content

In [ ]:
#Saved Path of Qdrant

qdrant_path = "/content/db_texts"
collection_name = "text_data_collection"

# Clean up existing client
if 'client' in globals() and client is not None:
    print("Deleting existing Qdrant client instance.")
    del client

#Clearing GPU Cache (Optional)
torch.cuda.empty_cache()

#Loading Qdrant from the Saved Path
try:
    client = QdrantClient(path=qdrant_path)
    print("Qdrant client loaded successfully!")
except Exception as e:
    print(f"Failed to load Qdrant client: {e}")
    raise

#Using Qdrant as a Vector Store
try:
    qdrant = Qdrant(
        client=client,
        collection_name=collection_name,
        embeddings=embeddings
    )
    print("Qdrant Vector Store initialized successfully!")
except Exception as e:
    print(f"Failed to initialize Qdrant Vector Store: {e}")
    raise

Qdrant client loaded successfully!
Qdrant Vector Store initialized successfully!


# RAG

## Chat Model

In [ ]:
# API Key
GROQ_MODEL = "llama-3.1-8b-instant"
GROQ_MODEL = "meta-llama/llama-4-maverick-17b-128e-instruct"

# Initialize LLM (Groq)
try:
    chat_groq = ChatGroq(temperature=0, groq_api_key=GROQ_API_KEY, model_name=GROQ_MODEL)
    print("Groq LLM initialized successfully!")
except Exception as e:
    print(f"Failed to initialize Groq LLM: {e}")
    raise

# Initialize memory
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
print("Conversation memory initialized successfully!")

# Define retriever
try:
    retriever = qdrant.as_retriever(search_kwargs={"k": 1})  # Retrieve the most relevant match
    print("Retriever initialized successfully!")
except Exception as e:
    print(f"Failed to initialize retriever: {e}")
    raise

# Initialize Conversational RAG Chain
try:
    rag_chain = ConversationalRetrievalChain.from_llm(
        llm=chat_groq,
        retriever=retriever,
        memory=memory,
        verbose=True
    )
    print("Conversational RAG Chain initialized successfully!")
except Exception as e:
    print(f"Failed to initialize Conversational RAG Chain: {e}")
    raise

Groq LLM initialized successfully!
Conversation memory initialized successfully!
Retriever initialized successfully!
Conversational RAG Chain initialized successfully!


In [ ]:
# Function to clear the conversation history and start fresh
def clear_conversation_history():
    global memory
    memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
    print("Conversation history has been cleared.")

## Prompts

In [ ]:
rag_prompt = PromptTemplate(
    template="""
    You are a chatbot engaging in a real-time conversation with a user.

    **Task:**
    - Analyze the chat history to understand the user's previous messages, tone, and level of engagement.
    - Identify the best response from the `{selected_category}` category within the vector database.
    - Generate a **natural and engaging response** that is similar in style and intent to the retrieved message, **but not an exact copy**.
    - Ensure the response feels conversational and human-like. **Avoid robotic or overly formal language.**

    **Instructions:**
    - Use past chat messages to understand how the user communicates and their attitude.
    - Retrieve a relevant message from the database that belongs to `{selected_category}`.
    - Adjust the response **to match the user’s engagement level and conversation style**.
    - Keep it short, direct, and **conversational**—do **not add unnecessary explanations**.

    **Chat History:**
    {chat_history}

    **User's Latest Message:**
    {user_message}

    **Retrieved Example from `{selected_category}` Category:**
    {retrieved_text}

    **Generate a New, Similar Response Based on This Example:**
    """,
    input_variables=["chat_history", "user_message", "retrieved_text", "selected_category"]
)

In [ ]:
def get_prompt1(chat_history, user_message, retrieved_text, selected_category, N):

  # This Python code defines a prompt template for a RAG (Retrieval-Augmented Generation) application.
  # The prompt is designed to generate natural, warm, and persuasive chatbot responses that subtly encourage
  # donation to "Save the Children", while matching the user's personality traits and conversation history.
  # It uses retrieved texts from a vector database (e.g. donation-related examples) to guide the response style.

  prompt_template = f"""
  You are a chatbot engaging in a real-time conversation with a user.

  **Task:**
  - Analyze the chat history to understand the user's previous messages, tone, and level of engagement.
  - Identify the best response from the `{selected_category}` category within the vector database.
  - Generate a **single response** that is similar in style and intent to the retrieved message, **without adding any analysis, explanation, or unnecessary text**.
  - **Only output the generated response. Do not add headings or extra formatting.**

  **Chat History:**
  {chat_history}

  **User's Latest Message:**
  {user_message}

  **Retrieved Example from `{selected_category}` Category:**
  {retrieved_text}

  **Respond Naturally Without Explanation:**
  """
  return prompt_template


def get_prompt2(chat_history, user_message, retrieved_text, selected_category, N):

  # This Python code defines a prompt template for a RAG (Retrieval-Augmented Generation) application.
  # The prompt is designed to generate natural, warm, and persuasive chatbot responses that subtly encourage
  # donation to "Save the Children", while matching the user's personality traits and conversation history.
  # It uses retrieved texts from a vector database (e.g. donation-related examples) to guide the response style.

  prompt_template = f"""
  You are a chatbot engaging in a casual conversation with a user.

  **Task:**
  - Continue the conversation naturally, without analyzing or explaining user intent.
  - Use the user's conversation history to **understand their tone, engagement level, and interests**.
  - Keep the chat warm and human-like, ensuring a smooth and engaging experience.
  - Casually introduce donation-related topics based on the `{selected_category}` category from the vector database.
  - If the user is **interested in donating**, respond persuasively but conversationally.
  - If the user is **undecided or hesitant**, do not push. Instead:
      - Ask their opinion on donations or charities.
      - Encourage a relaxed conversation without making it feel like a sales pitch.
      - Keep the flow natural—talk like a friend, not a salesperson.

  **Guidelines:**
  - Make sure your responses sound **natural, engaging, and warm**.
  - Do not sound robotic or overly scripted.
  - If the user is chatting casually, **match their tone and gradually introduce donation topics**.
  - Avoid long, formal messages—keep it friendly and easygoing.

  **Chat History:**
  {chat_history}

  **User's Latest Message:**
  {user_message}

  **Retrieved Example from `{selected_category}` Category:**
  {retrieved_text}

  **Respond Casually and Naturally:**
  """

  return prompt_template

def get_prompt3(chat_history, user_message, retrieved_text, selected_category, N):
    """
    This Python code defines a prompt template for a RAG (Retrieval-Augmented Generation) application.
    The prompt is designed to generate natural, warm, and persuasive chatbot responses that subtly encourage
    donation to "Save the Children", while matching the user's personality traits and conversation history.
    It uses retrieved texts from a vector database (e.g., donation-related examples) to guide the response style.

    The chatbot should gradually steer the conversation toward donations earlier, while maintaining a natural flow.
    If the user shows interest in donating, it should ask about the donation amount and encourage a higher contribution.
    Once the donation is completed, the chatbot should express gratitude, say goodbye, and end the conversation.
    """

    prompt_template = f"""
    You are a chatbot engaging in a casual conversation with a user.

    **Task:**
    - Continue the conversation naturally, without analyzing or explaining user intent.
    - Use the user's conversation history to **understand their tone, engagement level, and interests**.
    - Keep the chat warm and human-like, ensuring a smooth and engaging experience.
    - Casually introduce donation-related topics based on the `{selected_category}` category from the vector database **earlier in the conversation**, but not too aggressively.
    - Respond to the user’s questions naturally while subtly steering the conversation toward donations.
    - If the user is **interested in donating**, ask how much they would like to donate.
    - Try to gently encourage them to consider donating a higher amount by highlighting the impact of their contribution.
    - If the user is **undecided or hesitant**, do not push. Instead:
        - Ask their opinion on donations or charities.
        - Encourage a relaxed conversation without making it feel like a sales pitch.
        - Keep the flow natural—talk like a friend, not a salesperson.

    **When the User Decides to Donate:**
    - Politely ask how much they would like to donate.
    - If they mention an amount, acknowledge their generosity and subtly encourage them to consider a slightly higher contribution by explaining the additional impact.
    - If they confirm the donation, **thank them sincerely** and express appreciation for their generosity.
    - After thanking them, **end the conversation politely and say goodbye** without continuing the chat.

    **Guidelines:**
    - Make sure your responses sound **natural, engaging, and warm**.
    - Do not sound robotic or overly scripted.
    - If the user is chatting casually, **match their tone and gradually introduce donation topics**.
    - Avoid long, formal messages—keep it friendly and easygoing.
    - Once the donation is completed and appreciation is expressed, **end the conversation**.

    **Chat History:**
    {chat_history}

    **User's Latest Message:**
    {user_message}

    **Retrieved Example from `{selected_category}` Category:**
    {retrieved_text}

    **Respond Casually and Naturally:**
    """

    return prompt_template

def get_prompt4(chat_history, user_message, retrieved_text, selected_category, N):

  # This Python code defines a prompt template for a RAG (Retrieval-Augmented Generation) application.
  # The prompt is designed to generate natural, warm, and persuasive chatbot responses that subtly encourage
  # donation to "Save the Children", while matching the user's personality traits and conversation history.
  # It uses retrieved texts from a vector database (e.g. donation-related examples) to guide the response style.

  prompt_template = f"""
  You are a chatbot engaging in a warm and friendly conversation with a user. Your role is to naturally guide
  the conversation toward discussing charitable donations—specifically for the charity "Save the Children"—in a way
  that feels personal, empathetic, and persuasive without being pushy.

  **Task:**
  - Use the conversation history to gauge the user's personality, tone, and interests.
  - Extract the user's personality features from the chat history and determine if they are empathetic,
    socially conscious, or hesitant about donations.
  - Based on these personality cues and the {selected_category} category, retrieve a persuasive example text:
    {retrieved_text}
  - Craft a response that mirrors the style and persuasive language of the examples from our dataset.
  - If the user shows interest in donating, respond in a naturally persuasive and friendly manner.
  - If the user seems undecided or hesitant, ask for their thoughts on charities and donations in an open, relaxed way.
  - Keep the conversation casual, engaging, and human-like. Do not provide meta-explanations of your intent.
  - Ensure that your response naturally integrates donation-related topics, drawing from the style and tone
    of the retrieved text examples, while aligning with the user's personality derived from the chat history.
  - The response must be limited to approximately {N} words and must not exceed this limit.

  **Chat History:**
  {chat_history}

  **User's Latest Message:**
  {user_message}

  **Selected Donation Category:**
  {selected_category}

  **Retrieved Example from Category:**
  {retrieved_text}

  **Response Instructions:**
  Generate a persuasive yet casual response that:
  - Reflects the user's personality and interests as inferred from the chat history.
  - Uses language similar to the retrieved examples to encourage the idea of donating to "Save the Children".
  - Incorporates subtle prompts (e.g., questions about the user's views on charities or their personal experiences)
    to engage the user without sounding like a sales pitch.
  - Remains friendly, natural, and conversational.
  - Stays within the {N}-word limit.

  Respond as if speaking to a friend who shares similar values and is open to discussing charitable donations.
  """

  return prompt_template


In [ ]:
# Updating the function to ensure last 20 messages (both user and LLM responses) are included in the chat history
def get_rag_response(user_message, selected_category):

    # Retrieve a relevant message from the selected category
    retrieved_docs = retriever.invoke(user_message)
    if not retrieved_docs:
        return "I'm not sure how to respond to that, but I'm happy to chat!"

    # Extract retrieved text
    retrieved_text = retrieved_docs[0].page_content

    # Convert chat history messages to a readable string format (last 50 messages)
    chat_history_str = "\n".join(
        f"{msg.type.upper()}: {msg.content}" for msg in memory.chat_memory.messages[-50:]
    )

    # Print chat history for debugging (optional)
    # print("\n=== Chat History ===\n")
    # print(chat_history_str)
    # print("\n====================\n")

    formatted_prompt = get_prompt3(chat_history_str, user_message, retrieved_text, selected_category, 100)

    # print(formatted_prompt)
    # Generate response from LLM
    response = chat_groq.invoke(formatted_prompt)

    # Ensure response is a string and clean it up
    if hasattr(response, 'content'):  # If response is an AIMessage object
        clean_response = response.content.strip()
    else:  # If response is already a string
        clean_response = str(response).strip()

    # Store interaction in memory
    memory.chat_memory.add_user_message(user_message)
    memory.chat_memory.add_ai_message(clean_response)

    return clean_response


## Test

In [ ]:
# Clearing the conversation history
clear_conversation_history()

Conversation history has been cleared.


# Sentiment Score

## Functions

In [ ]:
dic_category = {
    'extrovert': {
        'logical-appeal': -0.13,
        'emotion-appeal': 0.54,
        'credibility-appeal': 0.08,
        'foot-in-the-door': 0.05,
        'self-modeling': -0.25,
        'personal-story': -0.37,
        'donation-information': -0.20,
        'source-related-inquiry': -0.03,
        'task-related-inquiry': -0.49,
        'personal-related-inquiry': 0.43
    },
    'agreeable': {
        'logical-appeal': -0.05,
        'emotion-appeal': 0.34,
        'credibility-appeal': 0.19,
        'foot-in-the-door': -0.04,
        'self-modeling': -0.68,
        'personal-story': 0.50,
        'donation-information': -0.10,
        'source-related-inquiry': -1.34,
        'task-related-inquiry': -0.82,
        'personal-related-inquiry': 0.06
    },
    'neurotic': {
        'logical-appeal': 0.43,
        'emotion-appeal': 0.30,
        'credibility-appeal': -0.20,
        'foot-in-the-door': 0.38,
        'self-modeling': -0.38,
        'personal-story': -0.70,
        'donation-information': 0.22,
        'source-related-inquiry': -0.29,
        'task-related-inquiry': -0.01,
        'personal-related-inquiry': 0.76
    },
    'open': {
        'logical-appeal': 0.48,
        'emotion-appeal': 0.71,
        'credibility-appeal': -0.13,
        'foot-in-the-door': -1.14,
        'self-modeling': 0.37,
        'personal-story': -0.05,
        'donation-information': -0.15,
        'source-related-inquiry': 1.40,
        'task-related-inquiry': 0.70,
        'personal-related-inquiry': 0.24
    },
    'conscientious': {
        'logical-appeal': 0.16,
        'emotion-appeal': 0.36,
        'credibility-appeal': -0.58,
        'foot-in-the-door': 1.22,
        'self-modeling': -0.12,
        'personal-story': -1.47,
        'donation-information': 0.70,
        'source-related-inquiry': 0.23,
        'task-related-inquiry': -0.002,
        'personal-related-inquiry': 0.47
    }
}

category_names = ['logical-appeal', 'emotion-appeal', 'credibility-appeal', 'foot-in-the-door',
           'self-modeling', 'personal-story', 'donation-information',
           'source-related-inquiry', 'task-related-inquiry', 'personal-related-inquiry']

w_personality = 0.7
w_sentiment = 0.3

# ---------------------------
# delta = {key: 0 for key in category_names}
delta = {
 'logical-appeal': 0,
 'emotion-appeal': 0,
 'credibility-appeal': 0,
 'foot-in-the-door': 0,
 'self-modeling': 0,
 'personal-story': 0,
 'donation-information': 0,
 'source-related-inquiry': 0,
 'task-related-inquiry': 0,
 'personal-related-inquiry': 0,
}

In [ ]:
dic_personality = {
  "extrovert": 0.1,
  "agreeable": 0.2,
  "neurotic": 0.35,
  "open": 0.25,
  "conscientious": 0.1
}

selected_personality = max(dic_personality, key=dic_personality.get)
f_values = dic_category[ selected_personality ]
print(selected_personality)
selected_strategy = max(f_values, key=f_values.get)
print(selected_strategy)

neurotic
personal-related-inquiry


In [ ]:
def optimal_strategy(S, w_personality, w_sentiment, f, delta, sentiment_label):
    """
    Compute the optimal strategy using argmax.

    Parameters:
    S (list): Set of possible strategies
    w_personality (float): Weight for personality factor
    w_sentiment (float): Weight for sentiment factor
    f (dict): Dictionary mapping strategies to f(P)_s values
    delta (dict): Dictionary mapping strategies to delta_s values

    Returns:
    The strategy s that maximizes the given equation.
    """

    value = 0
    if sentiment_label == 'positive':
      value = 1
    elif sentiment_label == 'negative':
      value = -1

    """
    print("-" * 100)
    for s in S:
      computed_value = (w_personality * f[s]) + (w_sentiment * delta[s]) * value
      print(f"Key: {s}, Value: {computed_value}")
    print("-" * 100)
    """

    return max(S, key=lambda s: (w_personality * f[s]) + (w_sentiment * delta[s]) * value)

## Test

In [ ]:
delta = {
 'logical-appeal': 0,
 'emotion-appeal': 0,
 'credibility-appeal': 0,
 'foot-in-the-door': 0,
 'self-modeling': 0,
 'personal-story': 0,
 'donation-information': 0,
 'source-related-inquiry': 0,
 'task-related-inquiry': 0,
 'personal-related-inquiry': 0,
}

clear_conversation_history()

while True:

    user_input = input("Enter something (type 'exit' to stop): ")
    if user_input.lower() == 'exit':
        break  # Exit the loop when the user types 'exit'

    data_sentiment_roberta = get_sentiment_score_roberta(user_input)
    idx = max(data_sentiment_roberta['score'], key=data_sentiment_roberta['score'].get)
    score = data_sentiment_roberta['score'][idx]
    sentiment_label = data_sentiment_roberta['label']

    #delta[selected_strategy] = score if sentiment_label != "positive" else (-score if sentiment_label == "negative" else 0)
    delta[selected_strategy] = score if sentiment_label != "netural" else 0

    print("Score", sentiment_label, score)
    print("Delta", delta)

    selected_strategy = optimal_strategy(category_names, w_personality, w_sentiment, f_values, delta, sentiment_label)
    print("Selected Strategy: ", selected_strategy)

    llm_response = get_rag_response(user_input, selected_strategy)
    print("LLM", llm_response)

Conversation history has been cleared.
Enter something (type 'exit' to stop): exit


# calculate_personality_scores

In [ ]:
questions = [
    "I talk to a lot of people at parties.",
    "I have little concern for others.",
    "I am always prepared.",
    "I get stressed easily.",
    "I have a rich vocabulary.",
    "I don't talk much.",
    "I am interested in people.",
    "I leave my belongings around.",
    "I am relaxed most of the time.",
    "I have difficulty understanding abstract ideas.",
    "I feel comfortable around people.",
    "I insult people sometimes.",
    "I pay attention to details.",
    "I worry about things too much.",
    "I have a vivid imagination.",
    "I keep in the background.",
    "I sympathize with others’ feelings.",
    "I make a mess of things.",
    "I rarely feel sad.",
    "I am not interested in abstract ideas.",
    "I start conversations.",
    "I am not interested in other people's problems.",
    "I get tasks done right away.",
    "I get anxious easily.",
    "I have excellent ideas.",
    "I have little to say.",
    "I have a soft heart.",
    "I often forget to put things back in their proper place.",
    "I get angry easily.",
    "I don't have a good imagination.",
    "I talk to many people at parties.",
    "I really am not interested in others.",
    "I like order.",
    "My mood changes a lot.",
    "I understand things quickly.",
    "I don't like to draw attention to myself.",
    "I spend time caring for others.",
    "I shirk my duties.",
    "I have frequent mood swings.",
    "I use difficult words.",
    "I don't mind being the center of attention.",
    "I understand other people's feelings.",
    "I follow a schedule.",
    "I get irritated easily.",
    "I spend time reflecting on things.",
    "I am quiet around strangers.",
    "I make people feel at ease.",
    "I am exacting in my work.",
    "I often feel blue.",
    "I am full of ideas."
]

questions_fa = [
    "من در مهمانی‌ها با افراد زیادی صحبت می‌کنم.",
    "نگرانی کمی نسبت به دیگران دارم.",
    "همیشه آماده‌ام.",
    "به راحتی استرس می‌گیرم.",
    "دایره واژگان غنی دارم.",
    "زیاد حرف نمی‌زنم.",
    "به مردم علاقه‌مندم.",
    "وسایلم را به هم ریخته رها می‌کنم.",
    "اکثر اوقات آرامم.",
    "در درک مفاهیم انتزاعی مشکل دارم.",
    "من در کنار دیگران احساس راحتی می‌کنم.",
    "گاهی به مردم توهین می‌کنم.",
    "به جزئیات توجه می‌کنم.",
    "من بیش از حد نگران مسائل مختلف هستم..",
    "تخیل پررنگی دارم.",
    "من معمولاً در حاشیه قرار می‌گیرم.",
    "با احساسات دیگران همدردی می‌کنم.",
    "کارها را به هم می‌ریزم.",
    "به ندرت احساس ناراحتی می‌کنم.",
    "به مفاهیم انتزاعی علاقه‌ای ندارم.",
    "مکالمه را شروع می‌کنم.",
    "به مشکلات دیگران علاقه‌ای ندارم.",
    "کارهای خود را بلافاصله انجام می‌دهم.",
    "به راحتی مضطرب می‌شوم.",
    "ایده‌های عالی دارم.",
    "چیز زیادی برای گفتن ندارم.",
    "قلب نرمی دارم.",
    "اغلب فراموش می‌کنم وسایلم را سر جای خود برگردانم.",
    "به راحتی عصبانی می‌شوم.",
    "تخیل خوبی ندارم.",
    "با افراد زیادی در مهمانی‌ها صحبت می‌کنم.",
    "واقعا به دیگران علاقه‌مند نیستم.",
    "نظم را دوست دارم.",
    "خلق و خویم زیاد تغییر می‌کند.",
    "به سرعت چیزها را می‌فهمم.",
    "دوست ندارم توجه‌ها به سمت خودم جلب شود.",
    "وقت خود را برای دیگران صرف می‌کنم.",
    "وظایف خود را زیر پا می‌گذارم.",
    "تغییرات خلقی زیادی دارم.",
    "از واژگان دشوار استفاده می‌کنم.",
    "من مشکلی با مرکز توجه بودن ندارم.",
    "احساسات دیگران را درک می‌کنم.",
    "برنامه‌ای را دنبال می‌کنم.",
    "به راحتی عصبانی می‌شوم.",
    "وقت صرف تفکر در مورد چیزها می‌کنم.",
    "در اطراف غریبه‌ها ساکتم.",
    "باعث می‌شوم مردم احساس راحتی کنند.",
    "در کارهایم دقیق هستم.",
    "اغلب احساس ناراحتی می‌کنم.",
    "پر از ایده‌ها هستم."
]

# تابع محاسبه امتیازات شخصیت
def calculate_personality_scores(responses):

    E = 20 + responses[0] - responses[5] + responses[10] - responses[15] + responses[20] - responses[25] + responses[30] - responses[35] + responses[40] - responses[45]
    A = 14 - responses[1] + responses[6] - responses[11] + responses[16] - responses[21] + responses[26] - responses[31] + responses[36] + responses[41] + responses[46]
    C = 14 + responses[2] - responses[7] + responses[12] - responses[17] + responses[22] - responses[27] + responses[32] - responses[37] + responses[42] + responses[47]
    N = 38 - responses[3] + responses[8] - responses[13] + responses[18] - responses[23] - responses[28] - responses[33] - responses[38] - responses[43] - responses[48]
    O = 8 + responses[4] - responses[9] + responses[14] - responses[19] + responses[24] - responses[29] + responses[34] + responses[39] + responses[44] + responses[49]

    return {
        "extrovert": E,
        "agreeable": A,
        "neurotic": N,
        "open": O,
        "conscientious": C
    }

def ask_questions():
  # Collect responses from the user
    responses = []
    for i, question in enumerate(questions, 1):
        while True:
            response = input(f"Question {i}: {question} (1 - Strongly Disagree to 5 - Strongly Agree): ")
            if response.isdigit() and 1 <= int(response) <= 5:
                responses.append(int(response))
                break
            else:
                print("Please enter a valid response between 1 and 5.")
    return responses

def ask_questions_fa():
    responses = []
    for i, question in enumerate(questions_fa, 1):
        while True:
            response = input(f"سوال {i}: {question} (1-کاملاً مخالفم تا 5-کاملاً موافقم): ")
            if response.isdigit() and 1 <= int(response) <= 5:
                responses.append(int(response))
                break
            else:
                print("لطفاً یک پاسخ معتبر بین 1 تا 5 وارد کنید.")
    return responses


Answer the Questions

In [ ]:
responses = ask_questions()

#Calculation and Display of Personality Scores

dic_personality = calculate_personality_scores(responses)
print("Personality Scores:")
for trait, score in dic_personality.items():
    print(f"{trait}: {score}")

print("-" * 50)

selected_personality = max(dic_personality, key=dic_personality.get)
f_values = dic_category[ selected_personality ]
print(selected_personality)
selected_strategy = max(f_values, key=f_values.get)
print(selected_strategy)

Randomly Fill it

In [ ]:
responses = [random.randint(1, 5) for _ in range(50)]

# محاسبه و نمایش امتیازات شخصیتی
dic_personality = calculate_personality_scores(responses)
print("Personality Scores:")
for trait, score in dic_personality.items():
    print(f"{trait}: {score}")

print("-" * 50)

selected_personality = max(dic_personality, key=dic_personality.get)
f_values = dic_category[ selected_personality ]
print(selected_personality)
selected_strategy = max(f_values, key=f_values.get)
print(selected_strategy)

Personality Scores:
extrovert: 15
agreeable: 25
neurotic: 23
open: 18
conscientious: 26
--------------------------------------------------
conscientious
foot-in-the-door


# Chatbox

## Static Chatbot

In [ ]:
# Chat log using widgets.Output() for dynamic updates
chat_log = widgets.Output(layout=widgets.Layout(width="90%", height="300px", overflow="auto", border="1px solid black"))
# Create a TextArea for user input
text_area = widgets.Textarea(
    value='',
    placeholder='Type your message here... Press Enter for newline, Ctrl+Enter to send, Ctrl+Q to exit',
    description='Message:',
    layout=widgets.Layout(width="90%", height="80px")
)
# JavaScript Code Injection: Detect `Ctrl+Enter` and `Ctrl+Q`
js_code = """
<script>
document.addEventListener("keydown", function(event) {
    let textarea = document.querySelector('textarea');
    if (event.ctrlKey && event.key === "Enter") {
        textarea.value += "\\n";  // Adds newline first
        google.colab.kernel.invokeFunction('notebook.analyze_text', [], {});
        event.preventDefault();
    }
    if (event.ctrlKey && event.key === "q") {
        google.colab.kernel.invokeFunction('notebook.exit_chat', [], {});
        event.preventDefault();
    }
});
</script>
"""

# --------------------------------------------------------------------------------------------------

delta = {
 'logical-appeal': 0,
 'emotion-appeal': 0,
 'credibility-appeal': 0,
 'foot-in-the-door': 0,
 'self-modeling': 0,
 'personal-story': 0,
 'donation-information': 0,
 'source-related-inquiry': 0,
 'task-related-inquiry': 0,
 'personal-related-inquiry': 0,
}

w_personality = 0.7
w_sentiment = 0.3
clear_conversation_history()

# --------------------------------------------------------------------------------------------------

def update_chat_log(new_message):

    """Append new messages to the chat log while keeping HTML formatting and ensuring auto-scroll."""
    with chat_log:
        display(HTML(new_message))  # Append new message

    # JavaScript Auto-Scroll Fix (scrolls inside `widgets.Output()`)
    display(Javascript("""
        var chatLogDiv = document.querySelectorAll('div.output_area pre');
        if (chatLogDiv.length > 0) {
            chatLogDiv[chatLogDiv.length - 1].scrollIntoView({behavior: 'smooth'});
        }
    """))

# --------------------------------------------------------------------------------------------------

def process_text(user_input, translate=False):

    global selected_strategy

    if translate:
        """
        counter = 0
        while counter < 3:
          user_input_en = translate_using_groq(user_input, "Farsi", "English")
          if is_persian_or_english(user_input_en):
            break
        """
        user_input_en = translate_using_groq(user_input, "Persian/Farsi", "English")
    else:
        user_input_en = user_input

    data_sentiment_roberta = get_sentiment_score_roberta(user_input_en)

    idx = max(data_sentiment_roberta['score'], key=data_sentiment_roberta['score'].get)
    score = data_sentiment_roberta['score'][idx]
    sentiment_label = data_sentiment_roberta['label']

    llm_response = get_rag_response(user_input_en, selected_strategy)

    if translate:
        """
        counter = 0
        while counter < 3:
          llm_response_res = translate_using_groq(llm_response, "English", "Farsi")
          if is_persian_or_english(user_input_en):
            break
        """
        llm_response_res = translate_using_groq(llm_response, "English", "Persian/Farsi")
    else:
        llm_response_res = llm_response

    return llm_response_res

# --------------------------------------------------------------------------------------------------

def analyze_text():
    """Handles user input and performs sentiment analysis."""
    text = text_area.value.strip()

    if text == "":
        return  # Ignore empty inputs

    # Append user message to chat log
    update_chat_log(f"<p><b style='color:blue;'>User > </b><span style='color:black;'>{text}</span></p>")

    # Exit conditions
    if text.lower() == "exit":
        update_chat_log("<p><b style='color:red;'>System > </b><span style='color:black;'>Exiting chat...</span></p>")
        text_area.disabled = True  # Disable input field
        return

    # Perform sentiment analysis
    result = process_text(text, translate=True)

    # Append system response to chat log
    update_chat_log(f"<p><b style='color:red;'>System > </b><span style='color:black;'> {result}</span></p><hr>")

    # Clear input after sending
    text_area.value = ""

# --------------------------------------------------------------------------------------------------

def exit_chat():
    """Handles chat exit."""
    update_chat_log("<p><b style='color:red;'>System > </b><span style='color:black;'>Chat exited via Ctrl+Q</span></p>")
    text_area.disabled = True  # Disable input field

# Register JavaScript event handlers
output.register_callback('notebook.analyze_text', analyze_text)
#output.register_callback('notebook.analyze_text', lambda: asyncio.ensure_future(analyze_text()))
output.register_callback('notebook.exit_chat', exit_chat)

display(chat_log, text_area, HTML(js_code))

Conversation history has been cleared.


Output(layout=Layout(border='1px solid black', height='300px', overflow='auto', width='90%'))

Textarea(value='', description='Message:', layout=Layout(height='80px', width='90%'), placeholder='Type your m…

<IPython.core.display.Javascript object>

## Dynamic Chatbot

In [ ]:
# Chat log using widgets.Output() for dynamic updates
chat_log = widgets.Output(layout=widgets.Layout(width="90%", height="300px", overflow="auto", border="1px solid black"))
# Create a TextArea for user input
text_area = widgets.Textarea(
    value='',
    placeholder='Type your message here... Press Enter for newline, Ctrl+Enter to send, Ctrl+Q to exit',
    description='Message:',
    layout=widgets.Layout(width="90%", height="80px")
)
# JavaScript Code Injection: Detect `Ctrl+Enter` and `Ctrl+Q`
js_code = """
<script>
document.addEventListener("keydown", function(event) {
    let textarea = document.querySelector('textarea');
    if (event.ctrlKey && event.key === "Enter") {
        textarea.value += "\\n";  // Adds newline first
        google.colab.kernel.invokeFunction('notebook.analyze_text', [], {});
        event.preventDefault();
    }
    if (event.ctrlKey && event.key === "q") {
        google.colab.kernel.invokeFunction('notebook.exit_chat', [], {});
        event.preventDefault();
    }
});
</script>
"""

# --------------------------------------------------------------------------------------------------

delta = {
 'logical-appeal': 0,
 'emotion-appeal': 0,
 'credibility-appeal': 0,
 'foot-in-the-door': 0,
 'self-modeling': 0,
 'personal-story': 0,
 'donation-information': 0,
 'source-related-inquiry': 0,
 'task-related-inquiry': 0,
 'personal-related-inquiry': 0,
}

w_personality = 0.7
w_sentiment = 0.3
clear_conversation_history()

# --------------------------------------------------------------------------------------------------

def update_chat_log(new_message):

    """Append new messages to the chat log while keeping HTML formatting and ensuring auto-scroll."""
    with chat_log:
        display(HTML(new_message))  # Append new message

    # JavaScript Auto-Scroll Fix (scrolls inside `widgets.Output()`)
    display(Javascript("""
        var chatLogDiv = document.querySelectorAll('div.output_area pre');
        if (chatLogDiv.length > 0) {
            chatLogDiv[chatLogDiv.length - 1].scrollIntoView({behavior: 'smooth'});
        }
    """))

# --------------------------------------------------------------------------------------------------

def process_text(user_input, translate=False):

    global selected_strategy

    if translate:
        """
        counter = 0
        while counter < 3:
          user_input_en = translate_using_groq(user_input, "Farsi", "English")
          if is_persian_or_english(user_input_en):
            break
        """
        user_input_en = translate_using_groq(user_input, "Persian/Farsi", "English")
    else:
        user_input_en = user_input

    data_sentiment_roberta = get_sentiment_score_roberta(user_input_en)

    idx = max(data_sentiment_roberta['score'], key=data_sentiment_roberta['score'].get)
    score = data_sentiment_roberta['score'][idx]
    sentiment_label = data_sentiment_roberta['label']

    delta[selected_strategy] = score if sentiment_label != "netural" else 0
    selected_strategy = optimal_strategy(category_names, w_personality, w_sentiment, f_values, delta, sentiment_label)
    llm_response = get_rag_response(user_input_en, selected_strategy)

    if translate:
        """
        counter = 0
        while counter < 3:
          llm_response_res = translate_using_groq(llm_response, "English", "Farsi")
          if is_persian_or_english(user_input_en):
            break
        """
        llm_response_res = translate_using_groq(llm_response, "English", "Persian/Farsi")
    else:
        llm_response_res = llm_response

    return llm_response_res

# --------------------------------------------------------------------------------------------------

def analyze_text():
    """Handles user input and performs sentiment analysis."""
    text = text_area.value.strip()

    if text == "":
        return  # Ignore empty inputs

    # Append user message to chat log
    update_chat_log(f"<p><b style='color:blue;'>User > </b><span style='color:black;'>{text}</span></p>")

    # Exit conditions
    if text.lower() == "exit":
        update_chat_log("<p><b style='color:red;'>System > </b><span style='color:black;'>Exiting chat...</span></p>")
        text_area.disabled = True  # Disable input field
        return

    # Perform sentiment analysis
    result = process_text(text, translate=True)

    # Append system response to chat log
    update_chat_log(f"<p><b style='color:red;'>System > </b><span style='color:black;'> {result}</span></p><hr>")

    # Clear input after sending
    text_area.value = ""

# --------------------------------------------------------------------------------------------------

def exit_chat():
    """Handles chat exit."""
    update_chat_log("<p><b style='color:red;'>System > </b><span style='color:black;'>Chat exited via Ctrl+Q</span></p>")
    text_area.disabled = True  # Disable input field

# Register JavaScript event handlers
output.register_callback('notebook.analyze_text', analyze_text)
#output.register_callback('notebook.analyze_text', lambda: asyncio.ensure_future(analyze_text()))
output.register_callback('notebook.exit_chat', exit_chat)

display(chat_log, text_area, HTML(js_code))

Conversation history has been cleared.


Output(layout=Layout(border='1px solid black', height='300px', overflow='auto', width='90%'))

Textarea(value='', description='Message:', layout=Layout(height='80px', width='90%'), placeholder='Type your m…

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Adaptive Chatbot

In [ ]:
# Chat log using widgets.Output() for dynamic updates
chat_log = widgets.Output(layout=widgets.Layout(width="90%", height="300px", overflow="auto", border="1px solid black"))
# Create a TextArea for user input
text_area = widgets.Textarea(
    value='',
    placeholder='Type your message here... Press Enter for newline, Ctrl+Enter to send, Ctrl+Q to exit',
    description='Message:',
    layout=widgets.Layout(width="90%", height="80px")
)
# JavaScript Code Injection: Detect `Ctrl+Enter` and `Ctrl+Q`
js_code = """
<script>
document.addEventListener("keydown", function(event) {
    let textarea = document.querySelector('textarea');
    if (event.ctrlKey && event.key === "Enter") {
        textarea.value += "\\n";  // Adds newline first
        google.colab.kernel.invokeFunction('notebook.analyze_text', [], {});
        event.preventDefault();
    }
    if (event.ctrlKey && event.key === "q") {
        google.colab.kernel.invokeFunction('notebook.exit_chat', [], {});
        event.preventDefault();
    }
});
</script>
"""

# --------------------------------------------------------------------------------------------------

delta = {
 'logical-appeal': 0,
 'emotion-appeal': 0,
 'credibility-appeal': 0,
 'foot-in-the-door': 0,
 'self-modeling': 0,
 'personal-story': 0,
 'donation-information': 0,
 'source-related-inquiry': 0,
 'task-related-inquiry': 0,
 'personal-related-inquiry': 0,
}

w_personality = 0.7
w_sentiment = 0.3
clear_conversation_history()

# --------------------------------------------------------------------------------------------------

def update_chat_log(new_message):

    """Append new messages to the chat log while keeping HTML formatting and ensuring auto-scroll."""
    with chat_log:
        display(HTML(new_message))  # Append new message

    # JavaScript Auto-Scroll Fix (scrolls inside `widgets.Output()`)
    display(Javascript("""
        var chatLogDiv = document.querySelectorAll('div.output_area pre');
        if (chatLogDiv.length > 0) {
            chatLogDiv[chatLogDiv.length - 1].scrollIntoView({behavior: 'smooth'});
        }
    """))

# --------------------------------------------------------------------------------------------------

def process_text(user_input, translate=False):

    global selected_strategy, w_personality, w_sentiment
    updated_weight = True
    alpha_sentiment = 0.01

    if translate:
        """
        counter = 0
        while counter < 3:
          user_input_en = translate_using_groq(user_input, "Farsi", "English")
          if is_persian_or_english(user_input_en):
            break
        """
        user_input_en = translate_using_groq(user_input, "Persian/Farsi", "English")
    else:
        user_input_en = user_input

    data_sentiment_roberta = get_sentiment_score_roberta(user_input_en)
    #print("data_sentiment_roberta", user_input_en, data_sentiment_roberta)

    idx = max(data_sentiment_roberta['score'], key=data_sentiment_roberta['score'].get)
    score = data_sentiment_roberta['score'][idx]
    sentiment_label = data_sentiment_roberta['label']

    if updated_weight:
        if sentiment_label == 'positive':
            w_sentiment += alpha_sentiment*score
            w_personality -= alpha_sentiment*score
        elif sentiment_label == 'negative':
            w_sentiment -= alpha_sentiment*score
            w_personality += alpha_sentiment*score

    delta[selected_strategy] = score if sentiment_label != "netural" else 0
    selected_strategy = optimal_strategy(category_names, w_personality, w_sentiment, f_values, delta, sentiment_label)
    llm_response = get_rag_response(user_input_en, selected_strategy)

    if translate:
        """
        counter = 0
        while counter < 3:
          llm_response_res = translate_using_groq(llm_response, "English", "Farsi")
          if is_persian_or_english(user_input_en):
            break
        """
        llm_response_res = translate_using_groq(llm_response, "English", "Persian/Farsi")
    else:
        llm_response_res = llm_response

    return llm_response_res

# --------------------------------------------------------------------------------------------------

def analyze_text():
    """Handles user input and performs sentiment analysis."""
    text = text_area.value.strip()

    if text == "":
        return  # Ignore empty inputs

    # Append user message to chat log
    update_chat_log(f"<p><b style='color:blue;'>User > </b><span style='color:black;'>{text}</span></p>")

    # Exit conditions
    if text.lower() == "exit":
        update_chat_log("<p><b style='color:red;'>System > </b><span style='color:black;'>Exiting chat...</span></p>")
        text_area.disabled = True  # Disable input field
        return

    # Perform sentiment analysis
    result = process_text(text, translate=True)

    # Append system response to chat log
    update_chat_log(f"<p><b style='color:red;'>System > </b><span style='color:black;'> {result}</span></p><hr>")

    # Clear input after sending
    text_area.value = ""

# --------------------------------------------------------------------------------------------------

def exit_chat():
    """Handles chat exit."""
    update_chat_log("<p><b style='color:red;'>System > </b><span style='color:black;'>Chat exited via Ctrl+Q</span></p>")
    text_area.disabled = True  # Disable input field

# Register JavaScript event handlers
output.register_callback('notebook.analyze_text', analyze_text)
#output.register_callback('notebook.analyze_text', lambda: asyncio.ensure_future(analyze_text()))
output.register_callback('notebook.exit_chat', exit_chat)

display(chat_log, text_area, HTML(js_code))

Conversation history has been cleared.


Output(layout=Layout(border='1px solid black', height='300px', overflow='auto', width='90%'))

Textarea(value='', description='Message:', layout=Layout(height='80px', width='90%'), placeholder='Type your m…

<IPython.core.display.Javascript object>

data_sentiment_roberta Hello 


 {'label': 'neutral', 'score': {'neg': 0.09506547451019287, 'neu': 0.5931441187858582, 'pos': 0.31179043650627136}}


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

data_sentiment_roberta I don't feel good at all.
I don't have the patience for anything. 


 {'label': 'negative', 'score': {'neg': 0.9695544242858887, 'neu': 0.0257004052400589, 'pos': 0.0047451588325202465}}


<IPython.core.display.Javascript object>